In [2]:
#The text that  has to be  summarized should be pasted in the file named 'text'
f=open('text.txt','r')

In [3]:
#breaking the text into sentances
text=[]
sentences=[]
for line in f:
    sentences=line.split('.')
    text.append(line)

In [4]:
text


['In its most general sense, the term "world" refers to the totality of entities, to the whole of reality or to everything that is.[1] The nature of the world has been conceptualized differently in different fields. Some conceptions see the world as unique while others talk of a "plurality of worlds". Some treat the world as one simple object while others analyze the world as a complex made up of many parts. In scientific cosmology the world or universe is commonly defined as "[t]he totality of all space and time; all that is, has been, and will be". Theories of modality, on the other hand, talk of possible worlds as complete and consistent ways how things could have been. Phenomenology, starting from the horizon of co-given objects present in the periphery of every experience, defines the world as the biggest horizon or the "horizon of all horizons". In philosophy of mind, the world is commonly contrasted with the mind as that which is represented by the mind. Theology conceptualizes 

In [5]:
sentences

['In its most general sense, the term "world" refers to the totality of entities, to the whole of reality or to everything that is',
 '[1] The nature of the world has been conceptualized differently in different fields',
 ' Some conceptions see the world as unique while others talk of a "plurality of worlds"',
 ' Some treat the world as one simple object while others analyze the world as a complex made up of many parts',
 ' In scientific cosmology the world or universe is commonly defined as "[t]he totality of all space and time; all that is, has been, and will be"',
 ' Theories of modality, on the other hand, talk of possible worlds as complete and consistent ways how things could have been',
 ' Phenomenology, starting from the horizon of co-given objects present in the periphery of every experience, defines the world as the biggest horizon or the "horizon of all horizons"',
 ' In philosophy of mind, the world is commonly contrasted with the mind as that which is represented by the mi

In [6]:
#importing libraries
import re
from nltk.tokenize import sent_tokenize

In [21]:
import nltk
nltk.download('stopwords')
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models import Word2Vec
from scipy  import spatial

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mithi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
#stopwords in nltk corpus
stopwords

<WordListCorpusReader in 'C:\\Users\\mithi\\AppData\\Roaming\\nltk_data\\corpora\\stopwords'>

In [11]:
#cleaning the sentences (removing stopwords and stemming)
corpus=[]
for item in sentences:
    sentance=re.sub('[^a-zA-Z]',' ',item)
    sentance=sentance.split()
    ps=PorterStemmer()
    stop_words=stopwords.words('english')
    sentance=[ps.stem(word) for word in sentance if word not in set(stop_words)]
    review=' '.join(sentance)
    corpus.append(review)
print(corpus)    

['in gener sens term world refer total entiti whole realiti everyth', 'the natur world conceptu differ differ field', 'some concept see world uniqu other talk plural world', 'some treat world one simpl object other analyz world complex made mani part', 'in scientif cosmolog world univers commonli defin total space time', 'theori modal hand talk possibl world complet consist way thing could', 'phenomenolog start horizon co given object present peripheri everi experi defin world biggest horizon horizon horizon', 'in philosophi mind world commonli contrast mind repres mind', 'theolog conceptu world relat god exampl god creation ident god two interdepend', 'in religion often tendenc downgrad materi sensori world favor spiritu world sought religi practic', 'a comprehens represent world place commonli found religion known worldview', 'cosmogoni field studi origin creation world eschatolog refer scienc doctrin last thing end world', '']


In [12]:
len(corpus)

13

In [13]:
#tokenizing each sentences
sentance_tokens=[[word for word in sentance.split(' ')] for sentance in corpus]

In [14]:
sentance_tokens

[['in',
  'gener',
  'sens',
  'term',
  'world',
  'refer',
  'total',
  'entiti',
  'whole',
  'realiti',
  'everyth'],
 ['the', 'natur', 'world', 'conceptu', 'differ', 'differ', 'field'],
 ['some',
  'concept',
  'see',
  'world',
  'uniqu',
  'other',
  'talk',
  'plural',
  'world'],
 ['some',
  'treat',
  'world',
  'one',
  'simpl',
  'object',
  'other',
  'analyz',
  'world',
  'complex',
  'made',
  'mani',
  'part'],
 ['in',
  'scientif',
  'cosmolog',
  'world',
  'univers',
  'commonli',
  'defin',
  'total',
  'space',
  'time'],
 ['theori',
  'modal',
  'hand',
  'talk',
  'possibl',
  'world',
  'complet',
  'consist',
  'way',
  'thing',
  'could'],
 ['phenomenolog',
  'start',
  'horizon',
  'co',
  'given',
  'object',
  'present',
  'peripheri',
  'everi',
  'experi',
  'defin',
  'world',
  'biggest',
  'horizon',
  'horizon',
  'horizon'],
 ['in',
  'philosophi',
  'mind',
  'world',
  'commonli',
  'contrast',
  'mind',
  'repres',
  'mind'],
 ['theolog',
  'conc

In [16]:
#training our Word2Vec Model 
model1 = Word2Vec(sentance_tokens, min_count = 1, 
                              vector_size = 1, window = 5)

In [17]:
#vectorising our sentances
tokens=[[model1.wv[word][0] for word in words]for words in sentance_tokens]

In [18]:
tokens

[[0.023247886,
  -0.39364037,
  0.6349584,
  0.07630539,
  -0.051768158,
  -0.82864773,
  0.09953679,
  -0.59314114,
  -0.47507164,
  0.50062376,
  -0.02995544],
 [-0.76582056,
  -0.19377422,
  -0.051768158,
  0.28781956,
  0.6459297,
  0.6459297,
  -0.4862234],
 [0.6567034,
  0.9239351,
  0.49815807,
  -0.051768158,
  0.96212375,
  -0.45174807,
  -0.15274553,
  -0.002714385,
  -0.051768158],
 [0.6567034,
  -0.43931848,
  -0.051768158,
  -0.8148006,
  -0.96027166,
  -0.37607193,
  -0.45174807,
  0.0043790606,
  -0.051768158,
  0.8081796,
  -0.23234248,
  -0.9446562,
  -0.73142743],
 [0.023247886,
  0.7317497,
  0.50684273,
  -0.051768158,
  0.6763823,
  0.9013626,
  0.8975486,
  0.09953679,
  -0.33883277,
  -0.09449977],
 [0.57717866,
  -0.75172454,
  -0.75124997,
  -0.15274553,
  -0.092978634,
  -0.051768158,
  0.9540182,
  -0.8752181,
  -0.41259506,
  -0.18119475,
  0.057885617],
 [-0.8919565,
  -0.26473898,
  0.5104065,
  -0.8751015,
  -0.08491067,
  -0.37607193,
  0.28288808,
  0.5

In [19]:
len(tokens)

13

In [22]:
#padding the tokens
max_len=max([len(words)for words in tokens])

tokens=[np.pad(words,(0,max_len-len(words)),'constant')for words in tokens]

In [23]:
tokens[1].size

16

In [24]:
max_len

16

In [25]:
#initializing the similarity martrix to zero
similarity_matrix=np.zeros([len(tokens),len(tokens)])
#calculating the spatial distance 
for i,row_embedding in enumerate(tokens):
    for j,column_embedding in enumerate(tokens):
        similarity_matrix[i][j]=1-spatial.distance.cosine(row_embedding,column_embedding)
        
        

In [26]:
#similarity matrix of sentances . value 1 for same sentance .
similarity_matrix

array([[ 1.00000000e+00, -2.95044452e-01,  1.26176909e-01,
         2.68943965e-01, -2.42299214e-01,  2.20348090e-01,
        -8.86183009e-02,  1.72083303e-02, -1.46867737e-01,
        -2.34956443e-01, -3.69692564e-01,  5.34741998e-01,
         1.60702784e-02],
       [-2.95044452e-01,  1.00000000e+00, -1.45589083e-01,
        -4.47241843e-01,  1.65143281e-01, -3.50087374e-01,
         7.06330687e-03,  2.30023116e-01, -2.55551696e-01,
         2.99444556e-01, -7.85968918e-03, -1.89115167e-01,
        -5.73940158e-01],
       [ 1.26176909e-01, -1.45589083e-01,  1.00000000e+00,
        -1.80190489e-01,  3.76348346e-01, -2.88418055e-01,
        -1.44839749e-01,  4.18620199e-01,  2.58350968e-02,
         4.17472601e-01, -6.73164353e-02, -2.74907380e-01,
         4.00285035e-01],
       [ 2.68943965e-01, -4.47241843e-01, -1.80190489e-01,
         1.00000000e+00, -4.63924766e-01,  1.02426782e-01,
        -9.99821350e-02, -8.65097269e-02, -8.13585371e-02,
        -2.95402050e-01,  3.51790816e

In [27]:
#initialising the text_rank to one
TextRank=np.ones([len(similarity_matrix[0])])

In [28]:
TextRank.reshape(13,)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [29]:
#calculating the text_rank of each sentance
for i in range(len(similarity_matrix[0])):
    for j in range(len(similarity_matrix[0])):
        if (j!=i):
            relativity=0.85*(TextRank[j]*similarity_matrix[i][j])
    TextRank[i]=(1-0.85)+relativity                  

In [30]:
TextRank

array([ 0.16365974, -0.33784913,  0.49024228,  0.40759628,  0.16139982,
        0.41685884, -0.19394746,  0.15974923,  0.46380505,  0.15974055,
        0.60256838,  0.10384924,  0.14520728])

In [31]:
#attaching textRank to respective sentance
top_sentance={sentance:TextRank[index] for index ,sentance in enumerate(sentences)}

In [32]:
#textRank of each sentance
top_sentance

{'In its most general sense, the term "world" refers to the totality of entities, to the whole of reality or to everything that is': 0.16365973660722377,
 '[1] The nature of the world has been conceptualized differently in different fields': -0.3378491342067718,
 ' Some conceptions see the world as unique while others talk of a "plurality of worlds"': 0.49024228006601334,
 ' Some treat the world as one simple object while others analyze the world as a complex made up of many parts': 0.4075962841510773,
 ' In scientific cosmology the world or universe is commonly defined as "[t]he totality of all space and time; all that is, has been, and will be"': 0.1613998181652278,
 ' Theories of modality, on the other hand, talk of possible worlds as complete and consistent ways how things could have been': 0.41685884445905685,
 ' Phenomenology, starting from the horizon of co-given objects present in the periphery of every experience, defines the world as the biggest horizon or the "horizon of all

In [33]:
#collecting top5 sentences
top=dict(sorted(top_sentance.items(), key=lambda x: x[1], reverse=True)[:5])

In [34]:
for sent in sentance:
    if sent in top.keys():
        print(sent)    

In [35]:
top

{' A comprehensive representation of the world and our place in it, as is commonly found in religions, is known as a worldview': 0.602568382024765,
 ' Some conceptions see the world as unique while others talk of a "plurality of worlds"': 0.49024228006601334,
 " Theology conceptualizes the world in relation to God, for example, as God's creation, as identical to God or as the two being interdependent": 0.4638050466775894,
 ' Theories of modality, on the other hand, talk of possible worlds as complete and consistent ways how things could have been': 0.41685884445905685,
 ' Some treat the world as one simple object while others analyze the world as a complex made up of many parts': 0.4075962841510773}